In [1]:
# -*- coding: utf-8 -*-

# Programmed  by Mingyu 'Ozone' CUI | DCOE, Sonepar APAC

import six
import packaging
import packaging.version
import packaging.specifiers
import packaging.requirements

import warnings
warnings.filterwarnings("ignore")

print("Warnings surpressed.")

import pandas as pd
import datetime

global null
null = '---'

global col_names
col_names = ['返修品', '已发货待收货', '可用库存总数', '在途PO', '欠料清零货期']

global indicator
indicator = '455'

# 冗余期设置 - 默认4，界面输入
global buffer_days
buffer_days = 4

# RANDOM COLOR GENERATOR FUNCTION

import colorsys
import random
 
def get_n_hls_colors(num):
    hls_colors = []
    i = 0
    step = 360.0 / num
    while i < 360:
        h = i
        s = 30 + random.random() * 10
        l = 30 + random.random() * 10
        _hlsc = [h / 360.0, l / 100.0, s / 100.0]
        hls_colors.append(_hlsc)
        i += step
 
    return hls_colors
 
def ncolors(num):
    rgb_colors = []
    if num < 1:
        return rgb_colors
    hls_colors = get_n_hls_colors(num)
    for hlsc in hls_colors:
        _r, _g, _b = colorsys.hls_to_rgb(hlsc[0], hlsc[1], hlsc[2])
        r, g, b = [int(x * 255.0) for x in (_r, _g, _b)]
        rgb_colors.append([r, g, b])
 
    return rgb_colors

def color(value):
    digit = list(map(str, range(10))) + list("ABCDEF")
    if isinstance(value, tuple):
        string = '#'
        for i in value:
            a1 = i // 16
            a2 = i % 16
            string += digit[a1] + digit[a2]
        return string
    elif isinstance(value, str):
        a1 = digit.index(value[1]) * 16 + digit.index(value[2])
        a2 = digit.index(value[3]) * 16 + digit.index(value[4])
        a3 = digit.index(value[5]) * 16 + digit.index(value[6])
        return (a1, a2, a3)

import tkinter as tk
import windnd
import openpyxl
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
import os
from PIL import Image, ImageTk

photo = Image.open('Hagemeyer.png')

'''BUNDLE FUNCTION: OUTPUT DATAFRAME PROCESS, INVENTORY ALLOCATION & DELIVERY DATE CALCULATION'''
def delivery_date_calc(df0, df1, df2, df3, df4):
    global null
    global indicator
    global col_names
    col = col_names[0]
    col_val = '返修品'
    shared_mat_cell = col_names[1:]
    delivered = col_names[-3:]
    spec_mat_str = ''
    
    # MOD TARGET DATAFRAME
    if df0.columns[-1] != col_names[-1]:
        for i in range(len(col_names)):
            len_col = df0.shape[1]
            df0.insert(len_col, col_names[i], value=None)
    df0['采购凭证'] = df0['采购凭证'].astype(str)
    df0['项目'] = df0['项目'].astype(str)
    df0['物料'] = df0['物料'].astype(str)
    df0['凭证日期'] = df0['凭证日期'].dt.date
    df0['交货日期'] = df0['交货日期'].dt.date
    
    #IDENTIFY & PROCESS 1-STOP MAT
    df0.loc[df0['采购凭证'].str.startswith(indicator), col] = col_val
    df0[col] = df0[col].fillna(null)
    df0.loc[df0[col]==col_val, df0.columns[-4:]] = null
    
    # PROCESS SHARED MATERIAL
    for i in df1.index:
        series = df1.iloc[i, 0]
        spec_mat_str += series + '|'
    spec_mat_str = spec_mat_str[:(len(spec_mat_str)-1)]
    df0.loc[~(df0['短文本'].str.contains(spec_mat_str))&(df0['欠料清零货期'].isnull()),'返修品'] = 'NDM'
    df0.loc[~(df0['短文本'].str.contains(spec_mat_str))&(df0['欠料清零货期'].isnull()),shared_mat_cell] = null
    
    # UNCONFIRMED DELIVERY
    df2['采购凭证'] = df2['采购凭证'].astype('str')
    df_temp = df0[['采购凭证', '短文本', '采购订单数量', '仍要交货数量']].copy(deep=True)
    for i in df0.index:
        df_temp.loc[i, '订货号'] = extract_modNum(df_temp.loc[i, '短文本'])
    
    df2.drop(['销售订单', '物料编号'], axis=1, inplace=True)
    df_mid = df2.groupby(['采购凭证', '订货号']).sum()
    df_temp.eval('key = 采购凭证 + 订货号', inplace=True)
    df_mid.eval('key = 采购凭证 + 订货号', inplace=True)
    df_temp = pd.merge(df_temp, df_mid, how='left',on='key')
    df_temp['已发货待收货'] = df_temp.apply(lambda x: (x.仍要交货数量 - (x.采购订单数量 - x.数量)), axis=1)
    df_temp.drop(df_temp.columns[:-1], axis=1, inplace=True)
    df0['已发货待收货'].fillna(df_temp['已发货待收货'], inplace=True)
    df0.loc[(df0['仍要交货数量'] == df0['已发货待收货']), delivered] = null
    
    df0['已发货待收货'].fillna(0, inplace=True)
    df0.loc[df0['已发货待收货']==null,'已发货待收货']=0
    df0['已发货待收货'] = df0['已发货待收货'].astype('int64')
    
    # EXTRACT MAT DICT & INVENTORY
    df_idic = pd.DataFrame()
    df_invtr = pd.DataFrame()
    df_wh = df3[['订货号', '库存']].copy(deep=True)
    df_wh = df_wh.groupby('订货号').sum()
    for i in df0.index:
        df_idic.loc[i, '订货号'] = extract_modNum(df0.loc[i, '短文本'])
    df_idic.drop_duplicates(inplace=True)
    df_idic.reset_index(drop=True, inplace=True)
    df_invtr = pd.merge(df_idic, df_wh, how='left', on='订货号')
    df_invtr['库存'].fillna(0, inplace=True)
    df_invtr.reset_index( inplace=True)
    mat_dict = dict(zip(df_invtr['订货号'], df_invtr['库存']))
    
    # FILL WITH EXISTING INVENTORY
    df_temp_invtr = df0[['短文本', '仍要交货数量', '已发货待收货','交货日期', '可用库存总数']].copy(deep=True)
    
    df_temp_invtr['订货号'] = df_temp_invtr.apply(lambda x: extract_modNum(x.短文本), axis=1)
    df_temp_invtr.drop(['短文本'], axis=1, inplace=True)
    for mat in mat_dict:
        if mat_dict[mat] > 0:
            df_temp_mat = df_temp_invtr.loc[df_temp_invtr['订货号']==mat].copy(deep=True)
            df_temp_mat.sort_values(by='交货日期',ascending=True, inplace=True)
            for index, row in df_temp_mat.iterrows():
                act_pending = df_temp_mat.loc[index, '仍要交货数量'] - df_temp_mat.loc[index, '已发货待收货']
                if row['可用库存总数'] != null:
                    gap = mat_dict[mat] - act_pending
                    if gap >=0:
                        df_temp_mat.loc[index, '可用库存总数'] = act_pending
                        df0.loc[index, '可用库存总数'] = df_temp_mat.loc[index, '可用库存总数']
                        df0.loc[index, '欠料清零货期'] = (datetime.datetime.now() + datetime.timedelta(days=buffer_days)).strftime('%Y-%m-%d')
                        mat_dict[mat] = gap
                    else:
                        df_temp_mat.loc[index, '可用库存总数'] = mat_dict[mat]
                        df0.loc[index, '可用库存总数'] = df_temp_mat.loc[index, '可用库存总数']
                        mat_dict[mat] = 0
    
    df0['可用库存总数'].fillna(null, inplace=True)
    df0.loc[df0['可用库存总数']==null,'可用库存总数']=0
    df0['可用库存总数'] = df0['可用库存总数'].astype('int64')
    df0.loc[(df0['仍要交货数量'] == df0['可用库存总数'] + df0['已发货待收货']), '在途PO'] = null
    
    # CHECK ENROUTE PO
    df_temp_PO = df4[['客户单号', '剩余交货数量（总计）', '确认到货日期（采购订单）', '制造商订货号']].copy(deep=True)
    df_temp_PO['确认到货日期（采购订单）'] = df_temp_PO['确认到货日期（采购订单）'].dt.date
    df_temp_target = df0[['采购凭证', '短文本', '仍要交货数量', '交货日期', '已发货待收货', '可用库存总数', '在途PO', '欠料清零货期']].copy(deep=True)
    df_temp_target['交货日期'] = pd.to_datetime(df_temp_target['交货日期'], format='%Y/%m/%d')
    for i in df0.index:
        df_temp_target.loc[i, '订货号'] = extract_modNum(df_temp_target.loc[i, '短文本'])
        
    df_emptyPO = df_temp_target.loc[(df_temp_target['在途PO']!=null)].copy(deep=True)
    df_emptyPO['在途PO'] = df_emptyPO['在途PO'].fillna(0)
    df_emptyPO['在途PO'].astype('int64')
    df_matDict = df_emptyPO['订货号']
    df_matDict.drop_duplicates(inplace=True)
    mat_PO_list = df_matDict.to_list()
    
    for mat in mat_PO_list:
        df_from = df_temp_PO.loc[df_temp_PO['制造商订货号']==mat]
        df_from.sort_values(by='确认到货日期（采购订单）',ascending=True, inplace=True)
        df_to = df_emptyPO.loc[df_emptyPO['订货号']==mat]
        df_to.sort_values(by='交货日期',ascending=True, inplace=True)
        
        while df_from['剩余交货数量（总计）'].sum()>0 and df_to.eval('仍要交货数量-已发货待收货-可用库存总数-在途PO').sum()>0:
            for i_to, r_to in df_to.iterrows():
                for i_from, r_from in df_from.iterrows():
                    po = r_to['仍要交货数量'] - r_to['已发货待收货'] - r_to['可用库存总数'] - r_to['在途PO']
                    if po >0 and df_from.loc[i_from, '剩余交货数量（总计）'] >= po:
                        r_to['在途PO'] += po
                        df_to.loc[i_to, '在途PO'] = r_to['在途PO']
                        df_temp_target.loc[i_to, '在途PO'] = df_to.loc[i_to, '在途PO']
                        df_from.loc[i_from, '剩余交货数量（总计）'] -= po
                        po = 0
                        if pd.isna(r_from['确认到货日期（采购订单）'])==False:
                            df_temp_target.loc[i_to, '欠料清零货期'] = (r_from['确认到货日期（采购订单）'] + datetime.timedelta(days=buffer_days)).strftime('%Y-%m-%d')
                        else:
                            df_temp_target.loc[i_to, '欠料清零货期'] = r_from['确认到货日期（采购订单）']
                    elif po >0 and df_from.loc[i_from, '剩余交货数量（总计）'] < po:
                        r_to['在途PO'] += r_from['剩余交货数量（总计）']
                        po -= r_from['剩余交货数量（总计）']
                        df_to.loc[i_to, '在途PO'] = r_to['在途PO']
                        df_from.loc[i_from, '剩余交货数量（总计）'] = 0
                    else:
                        print('>>>>> 该采购凭证欠料已清零 <<<<<')
    
    # FILL PO AMOUNT/DATE TO ORIGINAL TABLE
    df0['在途PO'].fillna(df_temp_target['在途PO'], inplace=True)
    df0['欠料清零货期'].fillna(df_temp_target['欠料清零货期'], inplace=True)
    
    # REPLACE ALL 0s TO '---'
    df0.已发货待收货[df0['已发货待收货']==0] = null
    df0.可用库存总数[df0['可用库存总数']==0] = null
    
    return df0

def color_execl(file_name):
    workbook = openpyxl.load_workbook(file_name, data_only=True)
    sheet1 = workbook.worksheets[0]
    for row in sheet1.rows:
        for cell in row:
            if cell.value is None or cell.value == 'NDM':
                orange_fill = PatternFill(fill_type='solid', fgColor="FFC125")
                cell.fill = orange_fill
    workbook.save(file_name)

global sheet_name
sheet_name = ''

# EXTRACT MODEL NUMBER FROM MATERIAL DESCRIPTION FUNCTION
def extract_modNum(desc):
    indicator = '\\'
    modNum = desc.split(indicator)[1]
    return modNum

def dragged_files(files):
    global sheet_name
    
    msg = ''.join((item.decode('gbk') for item in files))
    fp = msg.split('\n')[0]
    text0.insert('insert', fp)
    
    if sheet_name == '':
        df0 = pd.read_excel(fp)
    else:
        excel = load_workbook(fp)
        table = excel[sheet_name]
        data = table.values
        columns = next(data)[0:]
        df0 = pd.DataFrame(data, columns=columns)
        sheet_name = ''
    
    pd_list.append(df0)
    del df0
    return

class interface():
    def __init__(self, master, bg, ttl, btn_cap):
        global boo
        
        self.master = master
        self.master.config(bg=bg)
        self.master.title(ttl)
        self.master.geometry('600x400')
        self.interface = tk.Frame(self.master)
        self.interface.pack()
        self.label = tk.Label(self.interface, image = hag, text = 'Developed by:\n   Hagemeyer | DCOE Sonepar\n Jing CHEN | Ozone CUI    \n  Yan ZHANG | John ZHUANG', font=('Arial', 6), compound='left', width=800, height=38).pack(side='bottom', anchor='e')
        
        
        if counter == 0:
            btn_cmd = self.process
        elif counter == 1:
            boo = 1
            btn_cmd = self.next
        else:
            btn_cmd = self.next
        
        self.label = tk.Label(self.interface, text = '请确保每个导入Excel工作表的表头位于第一行；\n默认读取每个Excel工作簿第一张工作表；\n若需指定工作表名请在下方输入，点击“确认”按钮，然后拖放Excel文件到窗口', fg = '#10387d', bg='white', font=('Arial', 10), width=400, height=3)
        self.entry = tk.Entry(self.interface, show=None, font=('Arial', 12), width=30, justify=tk.CENTER)
        self.label.pack()
        self.entry.pack()
        btn_cpt = self.sheet_capture
        tk.Button(self.interface,text='确认', command=btn_cpt, width=10, height=1).pack(side='top')
        
        tk.Button(self.interface,text=btn_cap, font=('Arial', 12), command=btn_cmd, width=20, height=2).pack(side='bottom')
    
    def sheet_capture(self,):
        global sheet_name
        sheet_name = self.entry.get()
        print(sheet_name)
        return sheet_name
    
    def next(self,):
        global boo
        global counter
        global total
        self.interface.destroy()
        text0.delete(0.0, tk.END)
        counter -= 1
        num = total - counter
        interface(self.master, colors[num], sheet_names[num], button_caps[boo])
    
    def process(self,):
        
        for i in range(len(pd_list)):
            if isinstance(pd_list[i], pd.DataFrame):
                pd_list[i].replace('\s+', '', regex=True, inplace=True)
                pd_list[i] = pd_list[i].rename(columns=lambda x: x.replace('(', '').replace(')', '')).replace(' ', '')
        
        df_mid0 = pd_list[0].copy(deep=True)
        df_mid1 = pd_list[1].copy(deep=True)
        df_mid2 = pd_list[2].copy(deep=True)
        df_mid3 = pd_list[3].copy(deep=True)
        df_mid4 = pd_list[4].copy(deep=True)
        df_ddc = delivery_date_calc(df_mid0, df_mid1, df_mid2, df_mid3, df_mid4)
        
        export_path = os.path.join(os.path.expanduser('~'), 'Desktop') + '\\海格曼欠料表（完成）.xlsx'
        df_ddc.to_excel(export_path, index=False)
        
        color_execl(export_path)
        
        self.master.destroy()


# INSTANTIATE GUI

global counter
global total
global pd_list
global boo
sheet_names = ['0 海格曼欠料表', '1 达意隆专用物料查询表', '2 BI发货查询表', '3 BI库存库位查询表', '4 BI采购PO交期查询表']
colors = list(map(lambda x: color(tuple(x)), ncolors(len(sheet_names))))
counter = len(sheet_names) - 1
total  = len(sheet_names) - 1
print('counter = ', counter, 'total = ', total)
pd_list = []
button_caps = ['下一步 >>', '处理数据']
boo = 0

if __name__ == '__main__':
    root = tk.Tk()
    text0 = tk.Text(root, width=500, height=1, font=('Arial', 12))
    text0.pack(side='bottom')
    hag = ImageTk.PhotoImage(photo.resize((180, 30)))
    interface(root, colors[0], sheet_names[0], button_caps[0])
    windnd.hook_dropfiles(root, func=dragged_files)
    root.mainloop()

Warnings surpressed.
counter =  4 total =  4
贴标
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购凭证欠料已清零 <<<<<
>>>>> 该采购

| &#124; &#124; &#124; |                     | &#124; &#124; &#124; |
|--------------------------|-------------|---------------------------|
| &#124; &#124; &#124; | TEST AREA | &#124; &#124; &#124; |
| v   v   v |                                                      | v   v   v |

In [9]:
from PyInstaller.__main__ import run

if __name__ == '__main__':
    opts = ['-D', '-i',
            'D:\workTemp\Sonepar\HAG-DYL_Delivery_feedback\Ticalarc_Archery_128px_1075782_easyicon.net.ico', 
            'D:\workTemp\Sonepar\HAG-DYL_Delivery_feedback\Delivery_feedback_processor_v1.py']
    run(opts)

320814505 INFO: PyInstaller: 4.2
320814506 INFO: Python: 3.8.8 (conda)
320814508 INFO: Platform: Windows-10-10.0.19041-SP0
320814511 INFO: wrote C:\Users\Ozone Tsui\Documents\PY\Sonepar\Delivery_feedback_prototype\Delivery_feedback_processor_v1.spec
320814517 INFO: UPX is not available.
320814531 INFO: Extending PYTHONPATH with paths
['D:\\workTemp\\Sonepar\\HAG-DYL_Delivery_feedback',
 'C:\\Users\\Ozone Tsui\\Documents\\PY\\Sonepar\\Delivery_feedback_prototype']
320814536 INFO: checking Analysis
320814537 INFO: Building Analysis because Analysis-02.toc is non existent
320814539 INFO: Reusing cached module dependency graph...
320814734 INFO: Caching module graph hooks...
320814753 WARNING: Several hooks defined for module 'win32ctypes.core'. Please take care they do not conflict.
320814829 INFO: running Analysis Analysis-02.toc
320814832 INFO: Adding Microsoft.Windows.Common-Controls to dependent assemblies of final executable
  required by C:\Anaconda\anaconda3\envs\HAG-2103-YunyanZHA

320873129 INFO: Excluding import of PyQt5 from module PIL.ImageQt
320873149 INFO: Excluding import of tkinter from module PIL.ImageTk
320873153 INFO: Import to be excluded not found: 'FixTk'
320873158 INFO: Loading module hook 'hook-PIL.SpiderImagePlugin.py' from 'C:\\Anaconda\\anaconda3\\envs\\HAG-2103-YunyanZHANG\\lib\\site-packages\\PyInstaller\\hooks'...
320873177 INFO: Import to be excluded not found: 'FixTk'
320873179 INFO: Loading module hook 'hook-pkg_resources.py' from 'C:\\Anaconda\\anaconda3\\envs\\HAG-2103-YunyanZHANG\\lib\\site-packages\\PyInstaller\\hooks'...
320874788 WARNING: Hidden import "pkg_resources.py2_warn" not found!
320874792 WARNING: Hidden import "pkg_resources.markers" not found!
320874810 INFO: Excluding import of __main__ from module pkg_resources
320874815 INFO: Loading module hook 'hook-pygments.py' from 'C:\\Anaconda\\anaconda3\\envs\\HAG-2103-YunyanZHANG\\lib\\site-packages\\PyInstaller\\hooks'...
320878802 INFO: Loading module hook 'hook-pytz.py' from

320898528 INFO: checking PKG
320898530 INFO: Building PKG because PKG-01.toc is non existent
320898532 INFO: Building PKG (CArchive) PKG-01.pkg
320898684 INFO: Building PKG (CArchive) PKG-01.pkg completed successfully.
320898692 INFO: Bootloader C:\Anaconda\anaconda3\envs\HAG-2103-YunyanZHANG\lib\site-packages\PyInstaller\bootloader\Windows-64bit\run.exe
320898694 INFO: checking EXE
320898697 INFO: Building EXE because EXE-01.toc is non existent
320898700 INFO: Building EXE from EXE-01.toc
320898708 INFO: Copying icons from ['D:\\workTemp\\Sonepar\\HAG-DYL_Delivery_feedback\\Ticalarc_Archery_128px_1075782_easyicon.net.ico']
320898945 INFO: Writing RT_GROUP_ICON 0 resource with 20 bytes
320898947 INFO: Writing RT_ICON 1 resource with 67624 bytes
320898956 INFO: Appending archive to EXE C:\Users\Ozone Tsui\Documents\PY\Sonepar\Delivery_feedback_prototype\build\Delivery_feedback_processor_v1\Delivery_feedback_processor_v1.exe
320899127 INFO: Building EXE from EXE-01.toc completed successf